# Beta Diversity Analysis, Rarefaction and Significance Tests

        

## 0. Set_up

In [22]:
import os
import pandas as pd
import qiime2 as q2
from skbio import OrdinationResults
from qiime2 import Visualization
import matplotlib.pyplot as plt
from seaborn import scatterplot

%matplotlib inline

In [4]:
data_dir ='project_data'

In [3]:
ls

A_FirstLook.ipynb                figures/
B_Eating_habits.ipynb            G_Beta_Diversity.ipynb
C_Sequence_import.ipynb          H_DifferentialAbundance.ipynb
D_Taxonomy_NCBI.ipynb            I_Metagenomics.ipynb
D_Taxonomy_pre-classifier.ipynb  J_Functional_Redoundancy_and_Stability.ipynb
D_Taxonomy_SILVA.ipynb           project_data/
E_Phylogeny.ipynb                Z_GanttChart.ipynb
F_Alpha_Diversity.ipynb


In order to run this code, you need results of functions done on previous file : 
- `metadata` gathering is done in file A
- `diversity computation` is done in file F

<a id='sec1'></a>

## 1. Import metadata

Metadata of refers to students additional informations we collected into the file "cleaned_sample_meta_data.tsv".

In [14]:
df_meta = pd.read_csv(f'{data_dir}/sample_meta_data.tsv', sep='\t')

## 2.Compute diversity

Beta diversity measures the similarity between samples or groups of samples.        
To inspect groupings of beta diversity metrics across metadata categories, we will start by inspecting the principal coordinates (PCoA) plots created with the `qiime diversity core-metrics-phylogenetic` method in file F (directory core-metrics-results)

#### **Selection of categorical variables of interest**

In [23]:
df_cat = df_meta.select_dtypes(include=['object'])
df_cat.describe()

,GEN_age_cat,GEN_bmi_cat,GEN_collection_timestamp,GEN_country,GEN_geo_loc_name,GEN_host_common_name,GEN_last_move,GEN_last_travel,GEN_level_of_education,GEN_race,...,NUT_probiotic_frequency,NUT_red_meat_frequency,NUT_salted_snacks_frequency,NUT_seafood_frequency,NUT_sugary_sweets_frequency,NUT_vegetable_frequency,NUT_vitamin_b_supplement_frequency,NUT_vitamin_d_supplement_frequency,NUT_whole_eggs,NUT_whole_grain_frequency
count,523,523,523,523,523,523,523,523,523,523,...,523,523,523,523,523,523,523,523,523,523
unique,9,5,505,17,62,1,6,6,8,6,...,6,6,6,6,6,6,6,6,6,6
top,50s,Normal,2016-08-01 08:00:00,United Kingdom,United Kingdom:England,human,I have lived in my current state of residence ...,I have not been outside of my country of resid...,Graduate or Professional degree,Caucasian,...,Never,Occasionally,Rarely,Occasionally,Rarely,Daily,Never,Never,Occasionally,Regularly
freq,121,274,5,255,169,523,475,168,232,474,...,216,200,198,232,150,260,328,268,215,147


In [26]:
for col in df_cat.columns:
    print(col)

GEN_age_cat
GEN_bmi_cat
GEN_collection_timestamp
GEN_country
GEN_geo_loc_name
GEN_host_common_name
GEN_last_move
GEN_last_travel
GEN_level_of_education
GEN_race
GEN_sample_type
GEN_sex
NUT_alcohol_frequency
NUT_artificial_sweeteners
NUT_diet_type
NUT_drinks_per_session
NUT_fed_as_infant
NUT_fermented_plant_frequency
NUT_frozen_dessert_frequency
NUT_fruit_frequency
NUT_gluten
NUT_high_fat_red_meat_frequency
NUT_homecooked_meals_frequency
NUT_meat_eggs_frequency
NUT_milk_cheese_frequency
NUT_milk_substitute_frequency
NUT_olive_oil
NUT_poultry_frequency
NUT_prepared_meals_frequency
NUT_probiotic_frequency
NUT_red_meat_frequency
NUT_salted_snacks_frequency
NUT_seafood_frequency
NUT_sugary_sweets_frequency
NUT_vegetable_frequency
NUT_vitamin_b_supplement_frequency
NUT_vitamin_d_supplement_frequency
NUT_whole_eggs
NUT_whole_grain_frequency


Almost all the variables present in the dataframe df_cat can be interesting to test for beta-analysis. However, for the purpose of our research goals here, we will focus on data about age, BMI, country, level of education

#### **Permanova testing of categorical variables associations**

Associations between beta diversity and categorical variables can be statistically tested using a PERMANOVA test. This is a non-parametric statistical test that checks the null hypothesis that the distances between samples of one group are equivalent to distances to samples of another group. If this null hypothesis is rejected, we can infer that the distances between samples of one group differ significantly from the distances to samples in at least one other group. We can perform a PERMANOVA test checking whether the observed categories are significantly grouped in QIIME 2 with the `qiime diversity beta-group-significance` method: 

In [40]:
os.mkdir('/home/jovyan/HealthyFood/JupyterDocs/project_data/core-metrics-results-bd')

**Bray curtis emperor**

In [53]:
! qiime diversity beta-group-significance \
    --i-distance-matrix $data_dir/core-metrics-results/bray_curtis_distance_matrix.qza \
    --m-metadata-file $data_dir/sample_meta_data.tsv \
    --m-metadata-column GEN_age_cat \
--p-pairwise\
--o-visualization $data_dir/core-metrics-results-bd/bc-age-significance.qzv

Saved Visualization to: project_data/core-metrics-results-bd/bc-age-significance.qzv


In [54]:
Visualization.load(f'{data_dir}/core-metrics-results-bd/bc-age-significance.qzv')

<visualization: Visualization uuid: 0472624f-796b-4cda-91a7-6afcb78559dc>

**Weighted unifrac emperor**

In [55]:
! qiime diversity beta-group-significance \
    --i-distance-matrix $data_dir/core-metrics-results/weighted_unifrac_distance_matrix.qza \
    --m-metadata-file $data_dir/sample_meta_data.tsv \
    --m-metadata-column GEN_age_cat \
--p-pairwise \
--o-visualization $data_dir/core-metrics-results-bd/wu_age-significance.qzv

Saved Visualization to: project_data/core-metrics-results-bd/wu_age-significance.qzv


In [57]:
Visualization.load(f'{data_dir}/core-metrics-results-bd/wu_age-significance.qzv')

<visualization: Visualization uuid: 0d0360c7-e4ec-4417-9db6-788ffba9e581>

#### **c)** Adonis implementation pf PERMANOVA tests 

The `adonis` implementation of PERMANOVA (part of the r-vegan package) accepts a formula as input, which can consist of one or more independent terms. This might be useful for testing which covariates explain the most variation in our datasets.

In [7]:
! qiime diversity adonis \
    --i-distance-matrix $data_dir/core-metrics-results/jaccard_distance_matrix.qza \ #change distance matrix
    --m-metadata-file $data_dir/cleaned_sample_meta_data.tsv \
    --p-formula "treatment*block" \ # change variables as well as + or *
    --o-visualization $data_dir/core-metrics-results/AD-jaccard-treatXbloc.qzv

Usage: qiime diversity adonis [OPTIONS]

  Determine whether groups of samples are significantly different from one
  another using the ADONIS permutation-based statistical test in vegan-R.
  The function partitions sums of squares of a multivariate data set, and is
  directly analogous to MANOVA (multivariate analysis of variance). This
  action differs from beta_group_significance in that it accepts R formulae
  to perform multi-way ADONIS tests; beta_group_signficance only performs
  one-way tests. For more details, consult the reference manual available on
  the CRAN vegan page: https://CRAN.R-project.org/package=vegan

Inputs:
  --i-distance-matrix ARTIFACT
    DistanceMatrix     Matrix of distances between pairs of samples.
                                                                    [required]
Parameters:
  --m-metadata-file METADATA...
    (multiple          Sample metadata containing formula terms.
     arguments will    
     be merged)                                 

<a id='sec3'></a>